In [2]:
import pandas as pd
!pip install openpyxl
cdd= pd.read_excel('cooling.xlsx')

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [3]:
cdd.head()

,TIME,2013,2014,2015,2016,2017,2018,2019
0,BE10,2871.48,2138.45,2496,2556.95,2448.02,2383.35,2379.46
1,BE21,2844.27,2122.83,2438.23,2520.86,2400,2402.01,2362.91
2,BE22,2875.83,2157.03,2523.85,2581.59,2464.17,2430.43,2427.38
3,BE23,2787.88,2036.87,2390.52,2464.28,2350.29,2349.55,2345.61
4,BE24,2879.45,2161.24,2513.3,2573.89,2457.06,2396.26,2392.2


In [4]:

cdd_long = pd.melt(cdd, id_vars=['TIME'], var_name='year', value_name='cdd_value')
cdd_long['cdd_value'] = pd.to_numeric(cdd_long['cdd_value'], errors='coerce') # I force cdd_value to numeric
eu_mean = cdd_long['cdd_value'].mean() # I calculate mean and quantiles
q75 = cdd_long['cdd_value'].quantile(0.75)
q90 = cdd_long['cdd_value'].quantile(0.90)
cdd_long['cooling_above_eu_mean'] = (cdd_long['cdd_value'] > eu_mean).astype(int) #I create binary variables
cdd_long['cooling_above_q75'] = (cdd_long['cdd_value'] > q75).astype(int)
cdd_long['cooling_above_q90'] = (cdd_long['cdd_value'] > q90).astype(int)
print(cdd_long.head())


   TIME  year  cdd_value  cooling_above_eu_mean  cooling_above_q75  \
0  BE10  2013    2871.48                      1                  0   
1  BE21  2013    2844.27                      1                  0   
2  BE22  2013    2875.83                      1                  0   
3  BE23  2013    2787.88                      1                  0   
4  BE24  2013    2879.45                      1                  0   

   cooling_above_q90  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  


In [5]:
final_combined = pd.read_csv('final_combined_dataset.csv')
cdd_long = cdd_long.rename(columns={'TIME': 'group_id'})

# Be sure 'year' is numeric
cdd_long['year'] = pd.to_numeric(cdd_long['year'])
final_combined['year'] = pd.to_numeric(final_combined['year'])

# Merge
cooling = pd.merge(final_combined, cdd_long, on=['group_id', 'year'], how='left')

# 6. View result
print(cooling.head())

  group_id                                             region  year  \
0     BE10  Région de Bruxelles-Capitale/Brussels Hoofdste...  2013   
1     BE21                                    Prov. Antwerpen  2013   
2     BE22                                 Prov. Limburg (BE)  2013   
3     BE23                              Prov. Oost-Vlaanderen  2013   
4     BE24                               Prov. Vlaams-Brabant  2013   

  gdp_per_capita  partyvote  validvote  eurosceptic_vote_share  ETS_price  \
0          56800        NaN        NaN                     NaN       4.83   
1          36800        NaN        NaN                     NaN       4.83   
2          25700        NaN        NaN                     NaN       4.83   
3          28600        NaN        NaN                     NaN       4.83   
4          33900        NaN        NaN                     NaN       4.83   

   cdd_value  cooling_above_eu_mean  cooling_above_q75  cooling_above_q90  
0    2871.48                    1.

In [6]:
ghg= pd.read_excel('GHG.xlsx')

In [7]:
ghg_long = pd.melt(
    ghg,
    id_vars=['Country', 'NUTS 2', 'Sector'],
    var_name='year',
    value_name='ghg_value'
)
ghg_long['year'] = ghg_long['year'].str.replace('Y_', '').astype(int) # I Clean the 'year' column (remove "Y_" and convert to int)
ghg_pivot = ghg_long.pivot_table(
    index=['NUTS 2', 'year'],
    columns='Sector',
    values='ghg_value'
).reset_index()
ghg_pivot['ghg_total'] = ghg_pivot['Energy'] + ghg_pivot['Industry'] # I calculate total GHG (sum of Energy + Industry)
eu_mean = ghg_pivot['ghg_total'].mean() # I calculate EU-wide thresholds
q75 = ghg_pivot['ghg_total'].quantile(0.75)
q90 = ghg_pivot['ghg_total'].quantile(0.90)
ghg_pivot['ghg_above_eu_mean'] = (ghg_pivot['ghg_total'] > eu_mean).astype(int)
ghg_pivot['ghg_above_q75'] = (ghg_pivot['ghg_total'] > q75).astype(int)
ghg_pivot['ghg_above_q90'] = (ghg_pivot['ghg_total'] > q90).astype(int)
ghg_pivot = ghg_pivot.rename(columns={'NUTS 2': 'group_id'})

In [8]:
ghg_pivot.head()

Sector,group_id,year,Energy,Industry,ghg_total,ghg_above_eu_mean,ghg_above_q75,ghg_above_q90
0,AT11,2013,360.661920,251.154264,611.816183,0,0,0
1,AT11,2014,347.622203,246.472824,594.095027,0,0,0
2,AT11,2015,353.867074,250.948605,604.815679,0,0,0
3,AT11,2016,348.279739,249.947913,598.227652,0,0,0
4,AT11,2017,376.607682,252.850170,629.457852,0,0,0


In [9]:

ghg_pivot['year'] = ghg_pivot['year'].astype(int)
final_combined['year'] = final_combined['year'].astype(int)
merged = pd.merge(cooling, ghg_pivot, on=['group_id', 'year'], how='left')


In [10]:
print(merged.head())

  group_id                                             region  year  \
0     BE10  Région de Bruxelles-Capitale/Brussels Hoofdste...  2013   
1     BE21                                    Prov. Antwerpen  2013   
2     BE22                                 Prov. Limburg (BE)  2013   
3     BE23                              Prov. Oost-Vlaanderen  2013   
4     BE24                               Prov. Vlaams-Brabant  2013   

  gdp_per_capita  partyvote  validvote  eurosceptic_vote_share  ETS_price  \
0          56800        NaN        NaN                     NaN       4.83   
1          36800        NaN        NaN                     NaN       4.83   
2          25700        NaN        NaN                     NaN       4.83   
3          28600        NaN        NaN                     NaN       4.83   
4          33900        NaN        NaN                     NaN       4.83   

   cdd_value  cooling_above_eu_mean  cooling_above_q75  cooling_above_q90  \
0    2871.48                    1

In [11]:
mining= pd.read_excel('mining_units.xlsx')
mining.replace(":", pd.NA, inplace=True) # I replace ":" with NaN
mining_long = pd.melt(mining, id_vars=['TIME'], var_name='year', value_name='mining_value')
mining_long['year'] = pd.to_numeric(mining_long['year'], errors='coerce')
mining_long['mining_value'] = pd.to_numeric(mining_long['mining_value'], errors='coerce')
mining_long = mining_long.sort_values(by=['TIME', 'year']) # I sort for time-based filling
mining_long['mining_value'] = mining_long.groupby('TIME')['mining_value'].transform(lambda x: x.ffill().bfill())
mining_long = mining_long.rename(columns={'TIME': 'group_id'})

In [12]:
mining_long.head(20)

,group_id,year,mining_value
885,AT,2013.0,677.0
1421,AT,2014.0,692.0
1957,AT,2015.0,708.0
2493,AT,2016.0,701.0
3029,AT,2017.0,673.0
3565,AT,2018.0,640.0
4101,AT,2019.0,634.0
349,AT,NaN,634.0
886,AT1,2013.0,207.0
1422,AT1,2014.0,211.0


In [13]:
eu_mean_mining = mining_long['mining_value'].mean()
q75_mining = mining_long['mining_value'].quantile(0.75)
q90_mining = mining_long['mining_value'].quantile(0.90)
mining_long['mining_above_eu_mean'] = (mining_long['mining_value'] > eu_mean_mining).astype(int)
mining_long['mining_above_q75'] = (mining_long['mining_value'] > q75_mining).astype(int)
mining_long['mining_above_q90'] = (mining_long['mining_value'] > q90_mining).astype(int)
print(mining_long.head())


     group_id    year  mining_value  mining_above_eu_mean  mining_above_q75  \
885        AT  2013.0         677.0                     1                 1   
1421       AT  2014.0         692.0                     1                 1   
1957       AT  2015.0         708.0                     1                 1   
2493       AT  2016.0         701.0                     1                 1   
3029       AT  2017.0         673.0                     1                 1   

      mining_above_q90  
885                  1  
1421                 1  
1957                 1  
2493                 1  
3029                 1  


In [14]:
merged.to_excel('merged.xlsx', index=False)

In [15]:
merged['year'] = pd.to_numeric(merged['year'], errors='coerce')
mining_long['year'] = pd.to_numeric(mining_long['year'], errors='coerce')
merged = pd.merge(
    merged,
    mining_long[['group_id', 'year', 'mining_value', 'mining_above_eu_mean', 'mining_above_q75', 'mining_above_q90']],
    on=['group_id', 'year'],
    how='left'
)

In [16]:
print(merged.head(100))


   group_id                                             region  year  \
0      BE10  Région de Bruxelles-Capitale/Brussels Hoofdste...  2013   
1      BE21                                    Prov. Antwerpen  2013   
2      BE22                                 Prov. Limburg (BE)  2013   
3      BE23                              Prov. Oost-Vlaanderen  2013   
4      BE24                               Prov. Vlaams-Brabant  2013   
..      ...                                                ...   ...   
95     ES43                                        Extremadura  2013   
96     ES51                                           Cataluña  2013   
97     ES52                               Comunitat Valenciana  2013   
98     ES53                                      Illes Balears  2013   
99     ES61                                          Andalucía  2013   

   gdp_per_capita  partyvote  validvote  eurosceptic_vote_share  ETS_price  \
0           56800        NaN        NaN                  

In [17]:
tourists = pd.read_excel("tourists.xlsx")
tourists_long = pd.melt(
    tourists,
    id_vars=['TIME'],
    var_name='year',
    value_name='tourists'
)

tourists_long['year'] = pd.to_numeric(tourists_long['year'], errors='coerce')
tourists_long['tourists'] = pd.to_numeric(tourists_long['tourists'], errors='coerce')
tourists_long = tourists_long.sort_values(by=['TIME', 'year'])
tourists_long['tourists'] = tourists_long.groupby('TIME')['tourists'].transform(lambda x: x.ffill().bfill())
tourists_long = tourists_long.rename(columns={'TIME': 'group_id'})
eu_mean = tourists_long['tourists'].mean()
q75 = tourists_long['tourists'].quantile(0.75)
q90 = tourists_long['tourists'].quantile(0.90)
tourists_long['tourists_above_eu_mean'] = (tourists_long['tourists'] > eu_mean).astype(int)
tourists_long['tourists_above_q75'] = (tourists_long['tourists'] > q75).astype(int)
tourists_long['tourists_above_q90'] = (tourists_long['tourists'] > q90).astype(int)
print(tourists_long.head())


     group_id  year  tourists  tourists_above_eu_mean  tourists_above_q75  \
215      AT11  2013  684706.0                       0                   0   
543      AT11  2014  720509.0                       0                   0   
871      AT11  2015  729452.0                       0                   0   
1199     AT11  2016  796092.0                       0                   0   
1527     AT11  2017  792193.0                       0                   0   

      tourists_above_q90  
215                    0  
543                    0  
871                    0  
1199                   0  
1527                   0  


In [18]:
merged.to_excel('merged.xlsx', index=False)

In [19]:
merged['year'] = pd.to_numeric(merged['year'], errors='coerce')
tourists_long['year'] = pd.to_numeric(tourists_long['year'], errors='coerce')
merged_with_tourism = pd.merge(
    merged,
    tourists_long[['group_id', 'year', 'tourists', 'tourists_above_eu_mean', 'tourists_above_q75', 'tourists_above_q90']],
    on=['group_id', 'year'],
    how='left'
)
merged_with_tourism.to_excel("final_dataset_with_tourism.xlsx", index=False)

In [20]:
agri_gva = pd.read_excel("Agricultural_GVA.xlsx")
agri_gva = agri_gva.drop(columns=['Unnamed: 1'])
agri_gva.replace(":", pd.NA, inplace=True)
agri_long = pd.melt(
    agri_gva,
    id_vars=['TIME'],
    var_name='year',
    value_name='agri_gva'
)
agri_long['year'] = pd.to_numeric(agri_long['year'], errors='coerce')
agri_long['agri_gva'] = pd.to_numeric(agri_long['agri_gva'], errors='coerce')
agri_long = agri_long.sort_values(by=['TIME', 'year'])
agri_long['agri_gva'] = agri_long.groupby('TIME')['agri_gva'].transform(lambda x: x.ffill().bfill())
agri_long['agri_gva'] = agri_long['agri_gva'].fillna(0)
agri_long = agri_long.rename(columns={'TIME': 'group_id'})
eu_mean = agri_long['agri_gva'].mean()
q75 = agri_long['agri_gva'].quantile(0.75)
q90 = agri_long['agri_gva'].quantile(0.90)
agri_long['agri_above_eu_mean'] = (agri_long['agri_gva'] > eu_mean).astype(int)
agri_long['agri_above_q75'] = (agri_long['agri_gva'] > q75).astype(int)
agri_long['agri_above_q90'] = (agri_long['agri_gva'] > q90).astype(int)
print(agri_long.head())


     group_id  year  agri_gva  agri_above_eu_mean  agri_above_q75  \
187      AT11  2013    164.04                   0               0   
478      AT11  2014    158.66                   0               0   
769      AT11  2015    211.06                   0               0   
1060     AT11  2016    179.49                   0               0   
1351     AT11  2017    258.74                   0               0   

      agri_above_q90  
187                0  
478                0  
769                0  
1060               0  
1351               0  


In [21]:
merged_with_tourism['year'] = pd.to_numeric(merged_with_tourism['year'], errors='coerce')
agri_long['year'] = pd.to_numeric(agri_long['year'], errors='coerce')
final_merged = pd.merge(
    merged_with_tourism,
    agri_long[['group_id', 'year', 'agri_gva', 'agri_above_eu_mean', 'agri_above_q75', 'agri_above_q90']],
    on=['group_id', 'year'],
    how='left'
)

In [22]:
utility_vehicles = pd.read_excel("Utility_vehicles.xlsx")
utility_vehicles = utility_vehicles.drop(columns=['Unnamed: 1'])
utility_long = pd.melt(
    utility_vehicles,
    id_vars=['TIME'],
    var_name='year',
    value_name='utility_vehicles'
)
utility_long['year'] = pd.to_numeric(utility_long['year'], errors='coerce')
utility_long['utility_vehicles'] = pd.to_numeric(utility_long['utility_vehicles'], errors='coerce')
utility_long = utility_long.sort_values(by=['TIME', 'year'])
utility_long['utility_vehicles'] = utility_long.groupby('TIME')['utility_vehicles'].transform(lambda x: x.ffill().bfill())
utility_long['utility_vehicles'] = utility_long['utility_vehicles'].fillna(0)
utility_long = utility_long.rename(columns={'TIME': 'group_id'})
eu_mean = utility_long['utility_vehicles'].mean()
q75 = utility_long['utility_vehicles'].quantile(0.75)
q90 = utility_long['utility_vehicles'].quantile(0.90)
utility_long['vehicles_above_eu_mean'] = (utility_long['utility_vehicles'] > eu_mean).astype(int)
utility_long['vehicles_above_q75'] = (utility_long['utility_vehicles'] > q75).astype(int)
utility_long['vehicles_above_q90'] = (utility_long['utility_vehicles'] > q90).astype(int)
final_merged['year'] = pd.to_numeric(final_merged['year'], errors='coerce')
utility_long['year'] = pd.to_numeric(utility_long['year'], errors='coerce')
final_data = pd.merge(
    final_merged,
    utility_long[['group_id', 'year', 'utility_vehicles', 'vehicles_above_eu_mean', 'vehicles_above_q75', 'vehicles_above_q90']],
    on=['group_id', 'year'],
    how='left'
)
final_data.to_excel("final_data.xlsx", index=False)


In [1]:
data = pd.read_excel("final_data.xlsx")
eu_scores = data[data['year'].isin([2014, 2019])]
eu_pivot = eu_scores.pivot(index='group_id', columns='year', values='eurosceptic_vote_share').reset_index()
eu_pivot.columns.name = None
eu_pivot = eu_pivot.rename(columns={2014: 'score_2014', 2019: 'score_2019'})
eu_pivot['euroscepticism_change'] = eu_pivot['score_2019'] - eu_pivot['score_2014']
data = data.merge(eu_pivot[['group_id', 'euroscepticism_change']], on='group_id', how='left')
data.to_excel("data.xlsx", index=False)

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [9]:
df_votes = pd.read_excel("el_full_final_Eu.xlsx")
agg_votes['eurosceptic_vote_share'] = agg_votes['eurosceptic_vote_total'] / agg_votes['valid_vote_total']
pivoted = agg_votes.pivot(index='nuts2', columns='year', values='eurosceptic_vote_share').reset_index()
pivoted.columns.name = None  # Remove multi-index name
pivoted = pivoted.rename(columns={
    2014: 'vote_share_2014',
    2019: 'vote_share_2019'
})
pivoted['euroscepticism_change'] = pivoted['vote_share_2019'] - pivoted['vote_share_2014']
pivoted['vote_share_2014'] = (pivoted['vote_share_2014'] * 100).round(2)
pivoted['vote_share_2019'] = (pivoted['vote_share_2019'] * 100).round(2)
pivoted['euroscepticism_change'] = (pivoted['euroscepticism_change'] * 100).round(2)
print(pivoted[['nuts2', 'vote_share_2014', 'vote_share_2019', 'euroscepticism_change']])

    nuts2  vote_share_2014  vote_share_2019  euroscepticism_change
0    AT11            17.82            17.54                  -0.28
1    AT12            19.01            17.81                  -1.20
2    AT13            18.21            14.40                  -3.81
3    AT21            20.20            21.57                   1.37
4    AT22            24.24            19.74                  -4.50
..    ...              ...              ...                    ...
226  SE33             6.92            11.95                   5.03
227  SK01             8.41             9.11                   0.70
228  SK02             6.77            10.48                   3.71
229  SK03             7.99            11.73                   3.74
230  SK04             7.06             9.85                   2.79

[231 rows x 4 columns]
